In [ ]:
from conv_model import ConvProtein
from vocab import Vocab
from dataprocessor import *
from utils import *
import pandas as pd

In [ ]:
model = ConvProtein([21,32],[32,64,128,256,512],use_cuda = False)
model.load('./ckpt.21-32-32_64_128_256_512-100.pkl')

In [ ]:
x,y = get_data('./data/train_fixed.csv')
min_v,max_v = min(y),max(y)

def upscale(v , min_v, max_v):
    return ((v+1) / 2) * (max_v - min_v) + min_v

vocab = Vocab()
vocab.from_file()

test_data = pd.read_csv('./data/test.csv')

seq_ids   = test_data['seq_id'].values.tolist()
sequences = test_data['protein_sequence'].values.tolist()

In [ ]:
tms =[]
for batch in export_sequences(sequences,vocab,shape=[60,60]):
    out = model.predicts(batch)
    preds = [upscale(v,min_v,max_v) for v in out]
    tms.extend(preds)

In [9]:
data = pd.DataFrame(columns = ['seq_id','tm'])
data['seq_id'] = seq_ids
data['tm_p'] = tms

target = pd.read_csv('./data/sample_submission.csv')
target = target.merge(data,on='seq_id')

target.to_csv('submission.csv')